In [1]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint as fprint
import csv
import time

In [2]:
def get_author_url_set(author_name):
    cookies = {"Ecp_ClientId":"8181230144300598948","cnkiUserKey":"f7539957-65af-525e-faa1-06827035cf6a","UM_distinctid":"168901af2db2e5-0f9d022615f7b4-b781636-1fa400-168901af2dda1b","Ecp_IpLoginFail":"190131144.202.106.13","SID":"110105","CNZZDATA1257838113":"2120179519-1548925778-http%253A%252F%252Fsearch.cnki.net%252F%7C1548925778","KEYWORD":"%E7%89%9B%E4%B8%BD%E6%85%A7"}
    headers = {"Host":"yuanjian.cnki.net","Proxy-Connection":"keep-alive","Content-Length":"63","Cache-Control":"max-age=0","Origin":"http://yuanjian.cnki.net","Upgrade-Insecure-Requests":"1","DNT":"1","Content-Type":"application/x-www-form-urlencoded","User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36","Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8","Referer":"http://yuanjian.cnki.net/","Accept-Encoding":"gzip, deflate","Accept-Language":"zh-CN,zh;q=0.9,en;q=0.8"}
    base_url = "http://yuanjian.cnki.net/Search/Result"
    data = {"searchType":"MulityTermsSearch","Author":author_name}
    
    r = requests.post(base_url,cookies = cookies,headers = headers,data = data)
    bs = BeautifulSoup(r.text)
    res = bs.find_all("span",{"style":"color:red"})
    url_set = set()
    for item in res:
        url_set.add(item.parent["href"])
    return url_set          

In [3]:
def get_doc_url_set(author_url_set):
    url_set = set()
    for author_url in author_url_set:  
        r = requests.get(author_url)
        bs = BeautifulSoup(r.text)
        res = bs.find_all("p",{"class":"tit clearfix"})
        for item in res:
            url_set.add(item.a["href"])
    return url_set

In [4]:
def get_doc_bibilo(doc_url_set):
    res_file = open("res.csv","w",encoding="utf-8",newline="")
    csv_writer = csv.writer(res_file,delimiter="\t")
    
    for doc_url in doc_url_set:
        start_time = time.time()
        r = requests.get(doc_url)
        bs = BeautifulSoup(r.text)
        
        #journal
        try:
            journal = bs.find("b").text.strip()
        except:
            journal = " "

        #title
        title = bs.find("h1").text.strip()

        #author
        res = bs.find("div",{"style":"text-align:center; width:740px; height:30px;"})
        author_list = list()
        for item in res.find_all("a"):
            author_list.append(item.string.strip())

        #abstract
#         try:
#             abstract = bs.find("div",{"style":"text-align:left;word-break:break-all"}).font.next_sibling.string.strip()
#         except:
#             abstract = " "

        #address
        try:
            address = bs.find("div",{"style":"text-align:left;"}).a.string.strip()
        except:
            address = " "
            
        #keywords
        try:
            keyword_list = bs.find("meta",{"name":"keywords"})["content"].split()
        except:
            keyword_list = []

        #year
        try:
            year = bs.find("font",{"color":"#0080ff"}).string.strip()[:4]
        except:
            year = " "
        
        tmp = (title,";".join(author_list),journal,address,year,";".join(keyword_list))
        csv_writer.writerow(tmp)
        
        stop_time = time.time()
        print(stop_time-start_time)
    res_file.close()    

In [5]:
get_doc_bibilo(get_doc_url_set(get_author_url_set("陈云鹏")))

1.1751770973205566
0.6455004215240479
1.2787737846374512
0.9588937759399414
1.6322875022888184
0.5892360210418701
5.945256948471069
1.8604717254638672
0.4361391067504883
0.6968858242034912
1.2401072978973389
0.36912083625793457
0.5463294982910156
2.4473648071289062
0.5202267169952393
0.9956862926483154
0.369368314743042
0.613044023513794
1.354048252105713
1.8861851692199707
0.6010560989379883
0.9233815670013428
1.4283604621887207
1.20680832862854
1.568295955657959
1.040576457977295
0.556586503982544
1.0547797679901123
0.6670732498168945
1.6249656677246094
1.5145113468170166
0.5397062301635742
1.241008996963501
1.3452081680297852
1.7042434215545654
0.7736694812774658
0.3803064823150635
0.3920607566833496
1.0458168983459473
1.0356051921844482
1.3743340969085693
0.787644624710083
1.0322458744049072
1.682030200958252
0.6082432270050049
1.0063154697418213
2.6487395763397217
0.7737441062927246
0.5386538505554199
0.9666171073913574
1.473198413848877
0.6714041233062744
0.3962252140045166
1.421